<a href="https://colab.research.google.com/github/Nathan-Roll1/phone-benchmarking/blob/main/L2_Arctic_Ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install textgrid
import textgrid
import librosa
from IPython.display import Audio
import zipfile
import os

In [3]:
# !unzip /content/drive/MyDrive/l2arctic_release_v5.0.zip

In [4]:
# arctic_zips = [x for x in os.listdir() if '.zip' in x]

# for f in arctic_zips:
#     if os.path.exists(f):
#         with zipfile.ZipFile(f, 'r') as zip_ref:
#             zip_ref.extractall('/content/drive/Shareddrives/L2 Arctic')
#     else:
#         print(f"File {f} does not exist")

In [5]:
# from https://github.com/jhasegaw/phonecodes/blob/master/src/phonecode_tables.py

import re

_arpabet2ipa = {
    'AA':'ɑ',
    'AE':'æ',
    'AH':'ʌ',
    'AH0':'ə',
    'AO':'ɔ',
    'AW':'aʊ',
    'AY':'aɪ',
    'EH':'ɛ',
    'ER':'ɝ',
    'ER0':'ɚ',
    'EY':'eɪ',
    'IH':'ɪ',
    'IH0':'ɨ',
    'IY':'i',
    'OW':'oʊ',
    'OY':'ɔɪ',
    'UH':'ʊ',
    'UW':'u',
    'B':'b',
    'CH':'tʃ',
    'D':'d',
    'DH':'ð',
    'EL':'l̩ ',
    'EM':'m̩',
    'EN':'n̩',
    'F':'f',
    'G':'ɡ',
    'HH':'h',
    'JH':'dʒ',
    'K':'k',
    'L':'l',
    'M':'m',
    'N':'n',
    'NG':'ŋ',
    'P':'p',
    'Q':'ʔ',
    'R':'ɹ',
    'S':'s',
    'SH':'ʃ',
    'T':'t',
    'TH':'θ',
    'V':'v',
    'W':'w',
    'WH':'ʍ',
    'Y':'j',
    'Z':'z',
    'ZH':'ʒ'
}

_tone2ipa = {
    'eng' : { '0':'', '1':'ˈ', '2':'ˌ' }
}

_arpabet2ipa.update(_tone2ipa['eng'])

In [6]:
def map_arpabet_to_ipa(arpabet_list):
  ipa_list = []
  for phone in arpabet_list:
    if phone in _arpabet2ipa:
      ipa_list.append(_arpabet2ipa[phone])
    else:
      ipa_list.append(' ')
  return ipa_list

# Load Audio

In [7]:
from IPython.core.display import display

In [25]:
def disp(directory, fpath):

  with open(f'/content/drive/Shareddrives/L2 Arctic/{directory}/transcript/{fpath}.txt','r') as f:
    word_level = f.read()

  tg = textgrid.TextGrid.fromFile(f'/content/drive/Shareddrives/L2 Arctic/{directory}/textgrid/{fpath}.TextGrid')
  phone_tier = [x for x in tg.tiers if x.name == 'phones'][0]

  # get phone marks, remove tone numbers, remove empties
  a = [re.sub('[0-9]', '', x) for x in [x.mark for x in phone_tier] if len(x) > 0]

  phonetic = ''.join(map_arpabet_to_ipa(a))

  print(f'Transcript: {word_level}\nIPA: {phonetic}')

  file_path = f'/content/drive/Shareddrives/L2 Arctic/{directory}/wav/{fpath}.wav'
  y, sr = librosa.load(file_path, sr=None)
  display(Audio(data=y, rate=sr))

In [9]:
disp('ZHAA', 'arctic_b0539')

Transcript: You were making them talk shop Ruth charged him
IPA: juwɝmeɪkɪŋðɛm tɔkʃɑp ɹuθtʃɑɹdʒdhɪm


In [26]:
for f in os.listdir('/content/drive/Shareddrives/L2 Arctic/')[:3]:
  try:
  #disp first wav and assoc. audio from each speaker
    f_1 = os.listdir(f'/content/drive/Shareddrives/L2 Arctic/{f}/wav/')[0]
    fname = f_1.split('.wav')[0]
    print(f)
    disp(f, fname)
  except (IndexError, FileNotFoundError):
    print(f'{f} no files in gdrive')

HQTV
Transcript: Philip began to feel that he had foolishly overestimated his strength
IPA:  fɪlɪp biɡæntufil ðæthihæd fulɪʃli oʊvɝɛstʌmeɪtɪdhɪzstɹɛŋkθ


HJK
Transcript: A cry of joy burst from Philip's lips
IPA:  ʌkɹaɪʌvdʒɔɪbɝstfɹʌmfɪlɪpslɪps


ASI
Transcript: A cry of joy burst from Philip's lips
IPA:  ʌkɹaɪʌvdʒɔɪ bɝstfɹʌmfɪlʌpslɪps


# Model Benchmarking

In [11]:
# !pip install git+https://github.com/smfsamir/flowmason
# !pip install ipdb
# !pip install dill
# from flowmason import conduct, MapReduceStep, SingletonStep, load_artifact

!pip install git+https://github.com/dmort27/panphon
import panphon.distance

!pip install jiwer
from jiwer import cer

In [ ]:
from transformers import pipeline

phone_model_to_test = ['ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns',
                         'vitouphy/wav2vec2-xls-r-300m-timit-phoneme',
                         'speech31/wav2vec2-large-english-TIMIT-phoneme_v3']

pipes = {m: pipeline("automatic-speech-recognition", model=m) for m in phone_model_to_test}

In [23]:
feat_edit_distance = panphon.distance.Distance().feature_edit_distance

In [44]:
audio_path = '/content/drive/Shareddrives/L2 Arctic/ERMS/wav/arctic_a0001.wav'
ground_truth = 'ʌkɹaɪʌvdʒɔɪbɝstfɹʌmfɪlɪpslɪps'

# load textgrid
tg = textgrid.TextGrid.fromFile('/content/drive/Shareddrives/L2 Arctic/ERMS/textgrid/arctic_a0001.TextGrid')
phone_tier = [x for x in tg.tiers if x.name == 'phones'][0]

# get phone marks, remove tone numbers, remove empties
a = [re.sub('[0-9]', '', x) for x in [x.mark for x in phone_tier] if len(x) > 0]

ground_truth = ''.join(map_arpabet_to_ipa(a))

# apply each model to audio file
outs = {m:v(audio_path)['text'].replace(' ','') for m,v in pipes.items()}

# outputs transcript, phone edit dist., CER
outs = {m:[v, round(feat_edit_distance(ground_truth, v),5), round(cer(ground_truth, v), 5)] for m,v in outs.items()}

In [45]:
outs

{'ctaguchi/wav2vec2-large-xlsr-japlmthufielta-ipa1000-ns': ['awrorofde̞dɛnd͡ʑɒt͡ʂoalvfilipt͡stiosɛt͡sɛtjo',
  6.54167,
  0.75],
 'vitouphy/wav2vec2-xls-r-300m-timit-phoneme': ['aʊɾɑɹəfðɛəəðɛnjʊtɹeɪlffilɪpstilzəsɛtɝə',
  7.04167,
  0.525],
 'speech31/wav2vec2-large-english-TIMIT-phoneme_v3': ['aʊtərəvðɛmðɛnjʊrtreɪəlffɪlɪpsstilzətɪtər',
  8.89583,
  0.625]}